<a href="https://colab.research.google.com/github/Jaqueline-Massucheto/BITCOIN_Market/blob/main/nlp_i2a2_as.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

dataset = pd.read_csv('/content/drive/MyDrive/i2a2/train.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer , TfidfTransformer

from sklearn.linear_model import LogisticRegression

from sklearn.naive_bayes import MultinomialNB
from sklearn import svm

x_train,x_test,y_train,y_test = train_test_split(dataset['review_title'], dataset.rating, test_size=0.2, random_state=42, stratify = dataset.rating)


pipe_lr = Pipeline([('vect', CountVectorizer(max_features = 2500)),
                   ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(random_state=42, max_iter=1000))])


pipe_svm = Pipeline([('vect', CountVectorizer(max_features = 2500)),
                   ('tfidf', TfidfTransformer()),
            ('clf', svm.SVC(random_state=42))])


pipe_nb = Pipeline([('vect', CountVectorizer(max_features = 2500)),
                   ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])



grid_params_lr = [{'clf__penalty': ['l1', 'l2'],
        'clf__C': [1.0, 0.5, 10, 100, 1000],
        'clf__solver': ['liblinear']}] 

grid_params_svm = [{'clf__kernel': ['linear', 'rbf'], 
        'clf__C': [9,10]}]

grid_params_nb = [{'clf__C': [0.1, 10, 100]}]

In [5]:
# Construct grid searches
jobs = -1

LR = GridSearchCV(estimator=pipe_lr,
            param_grid=grid_params_lr,
            scoring='accuracy',
            cv=10) 


SVM = GridSearchCV(estimator=pipe_svm,
            param_grid=grid_params_svm,
            scoring='accuracy',
            cv=10,
            n_jobs=jobs)

NB = GridSearchCV(estimator= pipe_nb, 
            param_grid= grid_params_nb, 
            scoring= 'accuracy', 
            cv=10, 
            n_jobs = jobs)


grids = [LR,SVM,NB]

# Creating a dict for our reference
grid_dict = {0: 'Logistic Regression', 
        1: 'Suport Vector Machine',
        2: 'MultinomialNB'}

# Fit the grid search objects
print('Performing model optimizations...')
best_acc = 0.0
best_clf = 0
best_gs = ''
for idx, gs in enumerate(grids):
    print('\nEstimator: %s' % grid_dict[idx])
    gs.fit(x_train, y_train)
    print('Best params are : %s' % gs.best_params_)
    # Best training data accuracy
    print('Best training accuracy: %.3f' % gs.best_score_)
    # Predict on test data with best params
    y_pred = gs.predict(x_test)
    # Test data accuracy of model with best params
    print('Test set accuracy score for best params: %.3f ' % accuracy_score(y_test, y_pred))
    # Track best (highest test accuracy) model
    if accuracy_score(y_test, y_pred) > best_acc:
        best_acc = accuracy_score(y_test, y_pred)
        best_gs = gs
        best_clf = idx
print('\nClassifier with best test set accuracy: %s' % grid_dict[best_clf])

Performing model optimizations...

Estimator: Logistic Regression
Best params are : {'clf__C': 1.0, 'clf__penalty': 'l2', 'clf__solver': 'liblinear'}
Best training accuracy: 0.597
Test set accuracy score for best params: 0.601 

Estimator: Suport Vector Machine


KeyboardInterrupt: ignored